In [14]:
import numpy as np
import pandas as pd

import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

#metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#feature selection
from sklearn import svm


from info_gain import info_gain


import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def readCSVFile(filePath):
        with open(filePath, 'r', newline='') as csvfile:
            has_header = csv.Sniffer().has_header(csvfile.read(1024))
            csvfile.seek(0)  # Rewind.
            dialect = csv.Sniffer().sniff(csvfile.readline(), [',',';'])
            csvfile.seek(0) 
            reader = csv.reader(csvfile, dialect)
            if(has_header):
                next(reader)  # Skip header row.
            dataset = pd.DataFrame(reader)
        return dataset
        #print(filePath)

In [3]:
def readExcel(filePath):
    dataset = pd.read_excel(filePath)
    return dataset

In [4]:
def custom_csv(fname):
    if fname.endswith((".data", ".csv")):
        return readCSVFile(fname)
    elif fname.endswith((".xlsx")):
        return readExcel(fname)

In [5]:
def getLabels(filePath):
    try:
        flag = 0
        dataset = custom_csv(filePath)
        #dataset = pd.read_csv("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/iris/iris.data")
        n = dataset.iloc[:, -1].nunique(dropna=False)
        perc = dataset.iloc[:, -1].value_counts(normalize=True)*100
        if(len(perc) > len(dataset.iloc[:, 0].value_counts(normalize=True)*100)):  #checking whether 1st column is label
            n=dataset.iloc[:, 0].nunique(dropna=False)
            flag = 1
        if(flag == 1):
            return dataset.iloc[:, 0]
        else:
            return dataset.iloc[:,-1]
    except:
        print("Can not read last column items for", filePath)

In [6]:
def getindependentVariables(filePath):
    try:
        flag = 0
        dataset = custom_csv(filePath)
        #dataset = pd.read_csv("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/iris/iris.data")
        n = dataset.iloc[:, -1].nunique(dropna=False)
        perc = dataset.iloc[:, -1].value_counts(normalize=True)*100
        if(len(perc) > len(dataset.iloc[:, 0].value_counts(normalize=True)*100)):  #checking whether 1st column is label
            n=dataset.iloc[:, 0].nunique(dropna=False)
            flag = 1
        if(flag == 1):
            return dataset.iloc[:, 1:]
        else:
            return dataset.iloc[:,:-1]
    except:
        print("Can not independent variabless for", filePath)    

In [7]:
def selectFeatures(X, impFeatures, threshold):
    X_new = pd.DataFrame()
    for idx, value in enumerate(impFeatures):
        if(value > threshold):
            X_new = pd.concat((X_new, X.iloc[:, idx]), axis=1)
    return X_new    

In [8]:
def mutualInfo(X,Y):
    return mutual_info_classif(X,Y)

In [9]:
def gainRatio(X,Y):
    info_gain_ratio_values = []
    for idx, col in X.iteritems():
        info_gain_ratio_values.append(info_gain.info_gain_ratio(col.values, Y.values.tolist()))
    return info_gain_ratio_values

In [10]:
def uniqueClasses(Y):
    n = len(Y.unique())
    return len(Y.unique())

In [11]:
def classifyKNN(filePath, fs):
    dataset = custom_csv(filePath)
  #  dataset = pd.read_csv("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data")
    dataset.head()
    X = getindependentVariables(filePath)
    Y = getLabels(filePath)
    if(fs == "MI"):
      impF = mutualInfo(X,Y)
    elif(fs == "GR"):
      impF = gainRatio(X,Y)
    elif(fs == "mrmr"):
      impF = mrmr_classif(X, Y, K = 10)
    X_new = selectFeatures(X, impF, 0.368)
    print(X_new.shape[1])  # add this as a feature ; one of the evaluation criteria
    X_train,X_test,y_train,y_test=train_test_split(X_new,Y,test_size=0.25,random_state=0)
    neighbors = uniqueClasses(Y)
    knn=KNeighborsClassifier(n_neighbors=neighbors)
    knn.fit(X_train, y_train)
    y_pred= knn.predict(X_test)
    target_names = (list(range(uniqueClasses(Y))))
    for index, item in enumerate(target_names):
      target_names[index] = str(item)
   # TN = cnf_matrix.values.sum() - (FP + FN + TP) 
    #tp, fn, fp, tn = metrics.confusion_matrix(y_test, y_pred, labels = getLabels(filePath)).ravel()
    report = classification_report(y_test, y_pred,  target_names=target_names, output_dict=True)
    return pd.DataFrame(report)


In [ ]:
#report should be dataset specific - each output file is for a single dataset

,0,1,2,accuracy,macro avg,weighted avg, CA, FS, %features  -> such 4*9 rows in each file


In [12]:
from sklearn.feature_selection import *
#dataset = readCSVFile("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data")
#datasethead()
a = (classifyKNN("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data", "MI"))
print(a.transpose())

b = (classifyKNN("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data", "GR"))
#d = pd.read_csv("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data")
#u = uniqueClasses(label)
#tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

print(b.transpose())


7
              precision    recall  f1-score    support
0              0.933333  0.875000  0.903226  16.000000
1              0.818182  0.857143  0.837209  21.000000
2              0.375000  0.375000  0.375000   8.000000
accuracy       0.777778  0.777778  0.777778   0.777778
macro avg      0.708838  0.702381  0.705145  45.000000
weighted avg   0.780337  0.777778  0.778511  45.000000
9
              precision    recall  f1-score    support
0              0.933333  0.875000  0.903226  16.000000
1              0.818182  0.857143  0.837209  21.000000
2              0.375000  0.375000  0.375000   8.000000
accuracy       0.777778  0.777778  0.777778   0.777778
macro avg      0.708838  0.702381  0.705145  45.000000
weighted avg   0.780337  0.777778  0.778511  45.000000


In [13]:
listofFiles={}
classificationPerfomance={}
i=1
j=1
classificationAlgo = {'knn', 'c45', 'nb', 'svm'}
featureAlgo = {'relief', 'relieff', 'ig', 'gr', 'fcbf', 'mrmr', 'qbb', 'focus', 'sc'}
for path, subdirs, files in os.walk(os.getcwd()+'/datasets/numeric datasets'):
    for name in files:
        if name.endswith((".data", ".csv", ".xlsx")):
            listofFiles[name]=os.path.join(path, name)

perfMetrics =            
for eachFile in listofFiles:
    classificationPerfomance[eachFile] = {}
    for eachClassAlgo in classificationAlgo:
        j+=1
        classificationPerfomance[eachFile][eachClassAlgo] = {}
        for eachFSAlgo in featureAlgo:
            fs = "FS"+str(i)
            classificationPerfomance[eachFile][eachClassAlgo][eachFSAlgo] = {}
            i+=1
            if(eachClassAlgo == "kNN"):
                perfMetrics = classifyKNN(listofFiles[eachFile], eachFSAlgo)
            #classificationPerfomance[eachFile]['acc'] = 1

        i=1
    j=1
   

SyntaxError: invalid syntax (<ipython-input-13-214af4ab24f9>, line 12)

In [82]:
list(range(5))

[0, 1, 2, 3, 4]

In [60]:
from sklearn.feature_selection import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectPercentile, chi2

dataset = pd.read_csv("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data")
Y = dataset.iloc[:, :1]
X = dataset.iloc[:, 1:]
#Y = getLabels("/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/iris/iris.data")

importantFeaures = mutual_info_classif(X, Y)
print((importantFeaures))

X_new = SelectPercentile(mutual_info_classif, percentile=70).fit_transform(X, Y)

model = LogisticRegression(solver='lbfgs')
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

fit = model.fit(X_train, y_train)

print(X_new.shape)
print(X.shape)
y_red=fit.predict(X_test)
# import the metrics class
cnf_matrix = metrics.confusion_matrix(y_test, y_red)
cnf_matrix


[0.4544441  0.27522577 0.06626352 0.21696826 0.17967958 0.42101022
 0.67034758 0.11196006 0.29338146 0.560239   0.47914357 0.50396243
 0.54630688]
(177, 9)
(177, 13)


array([[15,  1,  0],
       [ 0, 19,  1],
       [ 0,  0,  9]])

In [44]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)


In [50]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [51]:
classifier = KNeighborsClassifier(n_neighbors=3,p=2,metric='euclidean')

In [52]:
classifier.fit(X_train,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [53]:
y_pred = classifier.predict(X_test)


In [54]:
conf_matrix = confusion_matrix(y_test,y_pred)
print(conf_matrix)

[[11  1  0]
 [ 0 10  0]
 [ 0  3  5]]


In [55]:
print(accuracy_score(y_test,y_pred))


0.8666666666666667


In [30]:
print(f1_score(y_test,y_pred, average="macro"))


0.8530286138981791


In [115]:
def classifaction_report_csv(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv('classification_report.csv', index = False)

In [118]:
url = '/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data'


data = pd.read_csv(url,low_memory=False)
x_1 = getindependentVariables(url)
data_dia = getLabels(url)
x_train, x_test, y_train, y_test = train_test_split(x_1, data_dia, test_size=0.3, random_state=42)
classifiers = [ 
KNeighborsClassifier(3),
DecisionTreeClassifier(),
RandomForestClassifier(),
AdaBoostClassifier(),
GradientBoostingClassifier(),
GaussianNB(),
LinearDiscriminantAnalysis() 
]

# Logging for Visual Comparison
log_cols = ["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(x_train, y_train)
    name = clf.__class__.__name__
    print("=" * 30)
    print(name)

    print('****Results****')
    train_predictions = clf.predict(x_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))

    train_predictions = clf.predict_proba(x_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))

    #log_entry = pd.DataFrame([[name, acc * 100, ll]], columns=log_cols)
    #log = log.append(log_entry)
    #report = classification_report(y_test, train_predictions)
    #print("log:",log)
    #print("=" * 30)



KNeighborsClassifier
****Results****
Accuracy: 74.0741%
Log Loss: 4.082993941999978
DecisionTreeClassifier
****Results****
Accuracy: 94.4444%
Log Loss: 1.918820910828373
RandomForestClassifier
****Results****
Accuracy: 100.0000%
Log Loss: 0.14604216622599353
AdaBoostClassifier
****Results****
Accuracy: 92.5926%
Log Loss: 1.1277569461761985
GradientBoostingClassifier
****Results****
Accuracy: 92.5926%
Log Loss: 0.4291264713030358
GaussianNB
****Results****
Accuracy: 100.0000%
Log Loss: 0.013696443178718095
LinearDiscriminantAnalysis
****Results****
Accuracy: 100.0000%
Log Loss: 0.003414563937484053


In [111]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [15]:
from mrmr import mrmr_classif


ModuleNotFoundError: No module named 'mrmr'

In [79]:
#https://github.com/shiralkarprashant/FCBF/blob/master/src/fcbf.py
def entropy(vec, base=2):
	" Returns the empirical entropy H(X) in the input vector."
	_, vec = np.unique(vec, return_counts=True)
	prob_vec = np.array(vec/float(sum(vec)))
	if base == 2:
		logfn = np.log2
	elif base == 10:
		logfn = np.log10
	else:
		logfn = np.log
	return prob_vec.dot(-logfn(prob_vec))


In [80]:
def conditional_entropy(x, y):
	"Returns H(X|Y)."
	uy, uyc = np.unique(y, return_counts=True)
	prob_uyc = uyc/float(sum(uyc))
	cond_entropy_x = np.array([entropy(x[y == v]) for v in uy])
	return prob_uyc.dot(cond_entropy_x)

In [81]:
def mutual_information(x, y):
	" Returns the information gain/mutual information [H(X)-H(X|Y)] between two random vars x & y."
	return entropy(x) - conditional_entropy(x, y)

In [82]:
def symmetrical_uncertainty(x, y):
	" Returns 'symmetrical uncertainty' (SU) - a symmetric mutual information measure."

	return 2.0*mutual_information(x, y)/(entropy(x) + entropy(y))


In [83]:
def getFirstElement(d):
	"""
	Returns tuple corresponding to first 'unconsidered' feature
	
	Parameters:
	----------
	d : ndarray
		A 2-d array with SU, original feature index and flag as columns.
	
	Returns:
	-------
	a, b, c : tuple
		a - SU value, b - original feature index, c - index of next 'unconsidered' feature
	"""

	t = np.where(d[:, 2] > 0)[0]
	if len(t):
		return d[t[0], 0], d[t[0], 1], t[0]
	return None, None, None

In [91]:
def c_correlation(X, y):
	"""
	Returns SU values between each feature and class.
	
	Parameters:
	-----------
	X : 2-D ndarray
		Feature matrix.
	y : ndarray
		Class label vector
		
	Returns:
	--------
	su : ndarray
		Symmetric Uncertainty (SU) values for each feature.
	"""
	su = np.zeros(X.shape[1])
	for i in np.arange(X.shape[1]):
		su[i] = symmetrical_uncertainty(X.iloc[:, i], y)

	return su

In [85]:
def getNextElement(d, idx):
	"""
	Returns tuple corresponding to the next 'unconsidered' feature.
	
	Parameters:
	-----------
	d : ndarray
		A 2-d array with SU, original feature index and flag as columns.
	idx : int
		Represents original index of a feature whose next element is required.
		
	Returns:
	--------
	a, b, c : tuple
		a - SU value, b - original feature index, c - index of next 'unconsidered' feature
	"""
	t = np.where(d[:, 2] > 0)[0]
	t = t[t > idx]
	if len(t):
		return d[t[0], 0], d[t[0], 1], t[0]
	return None, None, None


In [86]:
def removeElement(d, idx):
	"""
	Returns data with requested feature removed.
	
	Parameters:
	-----------
	d : ndarray
		A 2-d array with SU, original feature index and flag as columns.
	idx : int
		Represents original index of a feature which needs to be removed.
		
	Returns:
	--------
	d : ndarray
		Same as input, except with specific feature removed.
	"""
	d[idx, 2] = 0
	return d

In [106]:
def fcbf(X, y, thresh):
	"""
	Perform Fast Correlation-Based Filter solution (FCBF).
	
	Parameters:
	-----------
	X : 2-D ndarray
		Feature matrix
	y : ndarray
		Class label vector
	thresh : float
		A value in [0,1) used as threshold for selecting 'relevant' features. 
		A negative value suggest the use of minimum SU[i,c] value as threshold.
	
	Returns:
	--------
	sbest : 2-D ndarray
		An array containing SU[i,c] values and feature index i.
	"""
	n = X.shape[1]
	slist = np.zeros((n, 3))
	slist[:, -1] = 1

	# identify relevant features
	slist[:, 0] = c_correlation(X, y)  # compute 'C-correlation'
	idx = slist[:, 0].argsort()[::-1]
	slist = slist[idx, ]
	slist[:, 1] = idx
	if thresh < 0:
		thresh = np.median(slist[-1, 0])
		print("Using minimum SU value as default threshold: {0}".format(thresh))
	elif thresh >= 1 or thresh > max(slist[:, 0]):
		print("No relevant features selected for given threshold.")
		print("Please lower the threshold and try again.")
		exit()

	slist = slist[slist[:, 0] > thresh, :]  # desc. ordered per SU[i,c]

	# identify redundant features among the relevant ones
	cache = {}
	m = len(slist)
	p_su, p, p_idx = getFirstElement(slist)
	for i in range(m):
		p = int(p)
		q_su, q, q_idx = getNextElement(slist, p_idx)
		if q:
			while q:
				q = int(q)
				if (p, q) in cache:
					pq_su = cache[(p, q)]
				else:
					print(p)
					print(X[:,  p])
					#pq_su = symmetrical_uncertainty(X[:, p], X[:, q])
					#cache[(p, q)] = pq_su

				#if pq_su >= q_su:
				#	slist = removeElement(slist, q_idx)
				#q_su, q, q_idx = getNextElement(slist, q_idx)

		#p_su, p, p_idx = getNextElement(slist, p_idx)
		#if not p_idx:
		#	break

	#sbest = slist[slist[:, 2] > 0, :2]
	return NULL

In [42]:
from c45 import C45
from sklearn.datasets import load_iris

iris = load_iris()
clf = C45(attrNames=iris.feature_names)

TypeError: __init__() got an unexpected keyword argument 'attrNames'

In [45]:
from sklearn.naive_bayes import GaussianNB
X = getindependentVariables(url)
Y = getLabels(url)
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

gnb = GaussianNB()
target_names = (list(range(uniqueClasses(Y))))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)
pd.DataFrame(report).to_csv('sample.csv')
print(pd.DataFrame(report))


Accuracy: 0.9777777777777777
Number of mislabeled points out of a total 45 points : 1
              0          1         2  accuracy  macro avg  weighted avg
precision   1.0   1.000000  0.888889  0.977778   0.962963      0.980247
recall      1.0   0.952381  1.000000  0.977778   0.984127      0.977778
f1-score    1.0   0.975610  0.941176  0.977778   0.972262      0.978160
support    16.0  21.000000  8.000000  0.977778  45.000000     45.000000


In [52]:
def add_column_in_csv_2(input_file, output_file, transform_row, tansform_column_names):
    """ Append a column in existing csv using csv.reader / csv.writer classes"""
    # Open the input_file in read mode and output_file in write mode
    with open(input_file, 'r') as read_obj, \
            open(output_file, 'w', newline='') as write_obj:
        # Create a DictReader object from the input file object
        dict_reader = DictReader(read_obj)
        # Get a list of column names from the csv
        field_names = dict_reader.fieldnames
        # Call the callback function to modify column name list
        tansform_column_names(field_names)
        # Create a DictWriter object from the output file object by passing column / field names
        dict_writer = DictWriter(write_obj, field_names)
        # Write the column names in output csv file
        dict_writer.writeheader()
        # Read each row of the input csv file as dictionary
        for row in dict_reader:
            # Modify the dictionary / row by passing it to the transform function (the callback)
            transform_row(row, dict_reader.line_num)
            # Write the updated dictionary or row to the output file
            dict_writer.writerow(row)

In [66]:
#working column append n classification report
X = getindependentVariables(url)
Y = getLabels(url)
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

clf = svm.SVC(kernel='linear', random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
report = classification_report(y_test, y_pred,  target_names=target_names, output_dict=True)
#pd.DataFrame(report).to_csv('sample1.csv')
print(pd.DataFrame(report))
list_of_str = ['First', 'Second', 'Third', 'Fourth']
tail_len = 4

# The two steps in the description
n_rows = sum(1 for row in open('sample1.csv', 'r'))
df = pd.read_csv('sample1.csv', skiprows=range(1, n_rows - tail_len))
df_rest  = pd.read_csv('sample1.csv', skiprows=range(tail_len, n_rows))

print(df)
clsss = 'KNN'
percFeatures = 12
FSA = 'qwe'
df['class']=clsss
df['percF'] = percFeatures
df['FS'] = FSA
df
f = []
pd.concat([df_rest, df]).to_csv('sampe2.csv')

Accuracy: 0.9777777777777777
              0          1         2  accuracy  macro avg  weighted avg
precision   1.0   1.000000  0.888889  0.977778   0.962963      0.980247
recall      1.0   0.952381  1.000000  0.977778   0.984127      0.977778
f1-score    1.0   0.975610  0.941176  0.977778   0.972262      0.978160
support    16.0  21.000000  8.000000  0.977778  45.000000     45.000000
  Unnamed: 0     0          1         2  accuracy  macro avg  weighted avg
0  precision   1.0   1.000000  0.888889  0.977778   0.962963      0.980247
1     recall   1.0   0.952381  1.000000  0.977778   0.984127      0.977778
2   f1-score   1.0   0.975610  0.941176  0.977778   0.972262      0.978160
3    support  16.0  21.000000  8.000000  0.977778  45.000000     45.000000


In [41]:
url = '/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS/datasets/numeric datasets/wine/wine.data'
d = custom_csv(url)

X = getindependentVariables(url)
Y = getLabels(url)
impF = info_gain.info_gain_ratio(X, Y)
X_new = selectFeatures(X, impF, 0.368)
X_train,X_test,y_train,y_test=train_test_split(X_new,Y,test_size=0.25,random_state=0)
neighbors = uniqueClasses(Y)
knn=KNeighborsClassifier(n_neighbors=neighbors)
knn.fit(X_train, y_train)
y_pred= knn.predict(X_test)
target_names = (list(range(uniqueClasses(Y))))
for index, item in enumerate(target_names):
    target_names[index] = str(item)
   # TN = cnf_matrix.values.sum() - (FP + FN + TP) 
    #tp, fn, fp, tn = metrics.confusion_matrix(y_test, y_pred, labels = getLabels(filePath)).ravel()
report = classification_report(y_test, y_pred,  target_names=target_names, output_dict=True)
print(type(report))
 


ValueError: math domain error

In [17]:
from mrmr import mrmr_classif
mrmr_classif(X, Y, K = 10)

100%|██████████| 10/10 [00:02<00:00,  3.58it/s]


[6, 11, 12, 7, 13, 4, 8, 9, 1, 5]

In [18]:
from skrebate import ReliefF


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29269 sha256=d081a18c74d7342225ac604af14331b0eb4f8d4b0fbeb8d236ad685811b34a30
  Stored in directory: /home/d19125691/.cache/pip/wheels/86/52/36/f89dbee69f4529cd18215e030111c80b743b84205ea75f7df6
Successfully built skrebate
Note: you may need to restart the kernel to use updated packages.


In [29]:
from info_gain import info_gain

# Example of color to indicate whether something is fruit or vegatable
produce = ['apple', 'apple', 'apple', 'strawberry', 'eggplant']
fruit   = [ True  ,  True  ,  True  ,  True       ,  False    ]
colour  = ['green', 'green', 'red'  , 'red'       , 'purple'  ]

print(type(produce))

igr = info_gain.info_gain_ratio(fruit, colour)
igr1 = info_gain.info_gain_ratio(produce, colour)

print(igr1)

<class 'list'>
0.44220989760956225


In [19]:
pip install info_gain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
